In [2]:
import sys
sys.path.append(rf"/home/a/Lwy_research")
from Functions import *

In [5]:
date = "20250328"
# 实盘数据生成sh逐笔委托
trans_data_sh = pd.read_csv(rf"./example_data/sample_{date}_mdl_4_24_0.csv", index_col = False)
trans_data_sh = trans_data_sh.loc[trans_data_sh["SecurityID"].map(lambda x:str(x)[:2] in ['60', '68'])]

trans_data_sh["orderId"] = trans_data_sh[["BuyOrderNO", "SellOrderNO"]].max(axis= 1) 
cancel_data_sh = trans_data_sh.loc[trans_data_sh["Type"].isin(["D"])]
order_data_sh = trans_data_sh.loc[trans_data_sh["Type"].isin(["A", "T"])]

# 去掉集合竞价的成交单，包括早盘和尾盘，因为集合竞价成交单处理逻辑会有不同
order_data_sh = order_data_sh[~((order_data_sh["Type"] == "T") & ((order_data_sh["TickTime"] < "09:29:00.000") | (order_data_sh["TickTime"] > "14:59:00.000")))]

order_data_sh["Pricemax"] = order_data_sh["Price"]
order_data_sh["Pricemin"] = order_data_sh["Price"]

# 将sh订单还原，sh订单包含主动成交部分+剩余委托部分，需要通过groupby还原
order_data_sh = order_data_sh.groupby(["SecurityID", "orderId"]).agg({  'TickTime': 'first',
                                                    'TickBSFlag': 'first',
                                                    'Pricemax': 'max',
                                                    'Pricemin': 'min',
                                                    'BizIndex': 'min',
                                                    'Qty': 'sum'}).reset_index()
# 订单价格如果是买单，就取价格最大，卖单就取价格最小
order_data_sh["Price"] = (order_data_sh["TickBSFlag"] == "B") * order_data_sh['Pricemax'] + (order_data_sh["TickBSFlag"] == "S") * order_data_sh['Pricemin'] 

# SH需要的字段
local_data_sh = pd.DataFrame(index=order_data_sh.index)
local_data_sh['date'] = date
local_data_sh['code'] = order_data_sh['SecurityID'].map(lambda x:str(x).zfill(6))
local_data_sh['time'] = order_data_sh['TickTime'].map(lambda x:int(x.replace(':', '').replace('.', '')))
local_data_sh['orderId'] = order_data_sh['BizIndex'].astype("int")
local_data_sh['orderType'] = 'A'
local_data_sh['orderSide'] = order_data_sh['TickBSFlag']
local_data_sh['orderPrice'] = round(order_data_sh['Price'] + 1e-8, 2)
local_data_sh['orderVolume'] = order_data_sh['Qty'].astype("int")
local_data_sh['orderIdOrigin'] = order_data_sh['orderId'].astype("int")

local_cancel_sh = pd.DataFrame(index=cancel_data_sh.index)
local_cancel_sh['date'] = date
local_cancel_sh['code'] = cancel_data_sh['SecurityID'].map(lambda x:str(x).zfill(6))
local_cancel_sh['time'] = cancel_data_sh['TickTime'].map(lambda x:int(x.replace(':', '').replace('.', '')))
local_cancel_sh['orderId'] = cancel_data_sh['BizIndex'].astype("int")
local_cancel_sh['orderType'] = 'D'
local_cancel_sh['orderSide'] = cancel_data_sh['TickBSFlag']
local_cancel_sh['orderPrice'] = round(cancel_data_sh['Price'] + 1e-8, 2)
local_cancel_sh['orderVolume'] = -cancel_data_sh['Qty'].astype("int")
local_cancel_sh['orderIdOrigin'] = cancel_data_sh['orderId'].astype("int")

local_data_sh = pd.concat([local_data_sh, local_cancel_sh])
local_data_sh['orderAmount'] = round(local_data_sh['orderVolume'] *  local_data_sh['orderPrice']+ 1e-8, 2)
del trans_data_sh, order_data_sh, cancel_data_sh

# 同一时间按照orderid排序
local_data = local_data_sh.sort_values(["code", "time", "orderId"])